In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [7]:
runs = [
    # (  RUN NAME,                DF,                                              INFO  )
    # CPU baseline
    ("CPU", pd.read_csv("res/1.1.benchmark_results_cpu.csv", header=0), (1,)),
    ("CPU 1 Thread", pd.read_csv("res/6.benchmark_results_cpu_th_1.csv", header=0), (1,)),
    ("CPU 2 Thread", pd.read_csv("res/6.benchmark_results_cpu_th_2.csv", header=0), (2,)),
    ("CPU 4 Thread", pd.read_csv("res/6.benchmark_results_cpu_th_4.csv", header=0), (4,)),
    ("CPU 8 Thread", pd.read_csv("res/6.benchmark_results_cpu_th_8.csv", header=0), (8,)),
    ("CPU 12 Thread", pd.read_csv("res/6.benchmark_results_cpu_th_12.csv", header=0), (12,)),
    ("CPU 16 Thread", pd.read_csv("res/6.benchmark_results_cpu_th_16.csv", header=0), (16,)),
    # GPU optimizations
    ("GPU", pd.read_csv("res/1.2.benchmark_results_gpu.csv", header=0), (8704,)),
    ("GPU L1 opt", pd.read_csv("res/2.benchmark_results_gpu_cache_l1.csv", header=0), (8704,)),
    ("GPU No Branch", pd.read_csv("res/3.benchmark_results_gpu_no_branching.csv", header=0), (8704,)),
    ("GPU K Config", pd.read_csv("res/4.benchmark_results_gpu_config.csv", header=0), (8704,)),
    ("GPU Pre Increment", pd.read_csv("res/5.benchmark_results_gpu_pre_inc.csv", header=0), (8704,)),
    ("GPU 6", pd.read_csv("res/6.benchmark_results_gpu.csv", header=0), (8704,)),
    ("CPU Reduction", pd.read_csv("res/7.1.benchmark_results_cpu_th_16.csv", header=0), (16,)),
    ("CPU Critical", pd.read_csv("res/7.2.benchmark_results_cpu_th_16.csv", header=0), (16,)),
]

cpu_df = runs[0][1]
runs_gpu = runs[1:]

In [8]:
fig = go.Figure()
for name, df, _ in runs:
    fig.add_trace(go.Scatter(
        x=df["size"],
        y=df["avgEvolveTime"] / 1e6,  # time is originally in ns
        mode="lines+markers",
        name=name,
    ))
fig.update_layout(title="Average grid evolution time", xaxis_title='Lattice Size', yaxis_title="Time (ms)", width=500, height=400)
fig.update_yaxes(type="log")
fig.show()

In [9]:
fig = go.Figure()
for name, df, _ in runs_gpu:
    fig.add_trace(go.Scatter(
        x=df["size"],
        y=cpu_df["avgEvolveTime"] / df["avgEvolveTime"],
        mode="lines+markers",
        name=name,
    ))
fig.update_layout(title="Speedup", xaxis_title='Lattice Size', yaxis_title="Speedup", width=500, height=400)
fig.update_yaxes()
fig.show()

In [10]:
fig = go.Figure()
for name, df, info in runs_gpu:
    fig.add_trace(go.Scatter(
        x=cpu_df["size"],
        y=100 * (cpu_df["avgEvolveTime"] / df["avgEvolveTime"]) / info[0], # speedup / cores
        mode="lines+markers",
        name=name,
    ))
fig.update_layout(title="Efficiency", xaxis_title='Lattice Size', yaxis_title="Efficiency %", width=1000, height=600)
fig.update_yaxes()
fig.show()